# Annif Fusion experiment

**Osma Suominen, October 2018**

This notebook is an experiment for evaluating different methods to combine results of multiple subject indexing algorithms. We will use a document collection where gold standard subjects have been manually assigned and compare the subjects assigned by three different algorithm first individually, then in combinations created by different fusion approaches. In particular, we will test simple mean-of-subject-scores and union-of-top-K approaches as well as more advanced methods such as score normalization, isotonic regression and Learning to Rank style machine learning. Ideally, we would like to find a method for combining results from multiple algorithm that gives us the best quality results, combining the strengths of individual algorithms (both statistical/associative and lexical) while eliminating the effects of their weaknesses. The findings will then inform the further development of Annif.

The experiment was inspired by the Martin Toepfer's paper [Fusion architectures for automatic subject indexing under concept drift](https://link.springer.com/article/10.1007/s00799-018-0240-3) as well as discussions with him during and after the NKOS2018 workshop.

If you want to run this yourself, you need Python 3.5+ with the following libraries:

* jupyter (obviously)
* scikit-learn
* pyltr
* matplotlib (for the final plots)

This document is (c) Osma Suominen. It may be shared and reused according to the terms of the [CC Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/) license. Attribution should include the name of the author and a link to the original source document. However, code snippets in this notebook may be freely reused according to the [CC0 1.0 Universal](https://creativecommons.org/publicdomain/zero/1.0/) license. Attribution for code is requested but not required.

## Document corpus

We will use the ["Ask a Librarian" document corpus](https://github.com/NatLibFi/Annif-corpora/tree/master/fulltext/kirjastonhoitaja) for the experiment. It contains 3150 short question-answer pairs in Finnish language. The collection is subdivided into three subsets: train (n=2625), validate (n=213) and test (n=312). All the final evaluations will be performed on the test set, which contains the most recent questions asked during the year 2017. However, for some of the fusion methods we will make use of the train and validate subsets in order to fine-tune the way results are combined. The documents are represented as `*.txt` files.

All documents in the collection have been assigned gold standard subjects by librarians, stored in `*.tsv` files where the basenames correspond to the `*.txt` files. Librarians have assigned at least 4 subjects per document (average is 4.4-4.8 subjects per document, depending on the subset).

In addition, the documents have been analyzed using the [Annif](https://github.com/NatLibFi/Annif/) tool (development version v0.31.0) using three independent automated subject indexing algorithms: TF-IDF vector similarity, fastText and Maui, asking each algorithm to suggest up to 1000 subjects per document (however, we will in practice only load a fraction of these) with scores ranging from 1.0 to 0.0 given to each suggested subject. The subjects suggested by these algoritms are stored, respectively, in `*.tfidf`, `*.fasttext` and `*.maui` files.

First we load some basic modules and define the locations of the data files.

In [1]:
import pyltr
import os
import os.path
import glob
import numpy as np

SUBJECTFILE='data/yso.tsv'
TRAIN_DIR='data/kirjastonhoitaja/train/'
VALI_DIR='data/kirjastonhoitaja/validate/'
TEST_DIR='data/kirjastonhoitaja/test/'
ALGORITHMS = ('tfidf','fasttext','maui')  # these correspond to file extensions

## Subject vocabulary

All the subjects have been chosen from the Finnish General Ontology YSO, which contains around 28000 concepts.
Here we load the vocabulary from a TSV file where the first column is the concept URI and the second column is the concept label in Finnish. This has been extracted from the full YSO SKOS file.

Hereafter we will only use integer concept IDs which range from 0 to (n_concepts-1). All calculations are performed using the concept IDs so we just need to map the concept URIs which appear in files to their IDs.

In [2]:
uri_to_cid = {}
with open(SUBJECTFILE) as subjf:
    for cid, line in enumerate(subjf):
        uri = line.strip().split("\t")[0]
        uri_to_cid[uri] = cid
n_concepts = len(uri_to_cid)
print("Vocabulary loaded with", n_concepts, "concepts")

Vocabulary loaded with 27760 concepts


## Document corpus

We define a function to load the data from a directory of files in the format explained above. To be able to use both `scikit-learn` and `pyltr` libraries, we will need to express the data in two somewhat different formats, both based on multidimensional NumPy arrays:
    
1. pyltr ranking format, where the NumPy array rows correspond to document-subject pairs, i.e. there are multiple rows per document, one row per subject that has been suggested by at least one algorithm. A separate qids array ("query IDs", actually document IDs when applied this way) maps the document-subject pairs to the individual documents.
2. scikit-learn multilabel format, where the NumPy array has one row per document and the columns are individual concepts (either True/False for the gold standard subjects, or subject scores for the predicted subjects)

We will define this function once and then use it to parse all three document subsets (train, validate, test). We will only load up to 25 suggested subjects per document to keep the size of the rankings manageable.

In [11]:
def load_data(directory, max_pred=25):
    """Reads document corpus with gold standard and predicted subjects for each document.
    Only the top max_pred predictions per document and algorithm are considered.
    Returns a tuple with
     - predictions of individual algorithms as a 2D NumPy array with the shape (doc-suggestions, n_algos)
     - relevant (gold standard) concepts as a 1D NumPy array with the shape (doc-suggestions)
     - qids (actually document ids) as a 1D NumPy array with length doc-suggestions
     - gold standard subjects in sklearn multilabel format as a 2D NumPy array with shape (n_docs, n_concepts) 
     - concept IDs of suggested concepts as a 1D NumPy array with shape (doc-suggestions)
    """
    
    # temporary lists to be converted into NumPy arrays at the end
    docdatalist = []  
    docgoldlist = []
    doctruelist = []
    qidlist = []
    cidlist = []
    
    for fileid, txtfile in enumerate(glob.glob(os.path.join(directory, '*.txt'))):
        basename  = txtfile[:-4]
        
        # Initialize binary mask containing all the concepts that were suggested by any algorithm
        # This will be used to pare down the doc-suggestions for ranking so that we will skip concepts
        # that were not suggested by any algorithm. They will still be considered in F1 score evaluation,
        # which is not based only on the ranked suggestions but used the full gold standard subjects.
        mask = np.zeros(n_concepts, dtype=np.bool)

        docdata = np.zeros((n_concepts, len(ALGORITHMS)))
        for algoid, fileext in enumerate(ALGORITHMS):
            with open(basename + "." + fileext) as algooutput:
                lines = 0
                for line in algooutput:
                    uri, _, score = line.strip().split("\t")
                    if uri in uri_to_cid:
                        cid = uri_to_cid[uri]
                        docdata[cid,algoid] = float(score)
                        mask[cid] = True
                    lines += 1
                    if lines > max_pred:
                        break # read only top max_pred suggestions per algorithm
        docdatalist.append(docdata[mask])
    
        docgold = np.zeros(n_concepts, dtype=np.bool)
        with open(basename + ".tsv") as tsvfile:
            for line in tsvfile:
                uri = line.split("\t")[0]
                if uri in uri_to_cid:
                    cid = uri_to_cid[uri]
                    docgold[cid] = True
        docgoldlist.append(docgold[mask])
        doctruelist.append(docgold)
        
        qidlist.append(np.full(mask.sum(), fileid))
        cidlist.append(np.arange(n_concepts)[mask])

    return np.concatenate(docdatalist), np.concatenate(docgoldlist), np.concatenate(qidlist), \
           np.concatenate(cidlist), np.array(doctruelist)

# now load the train, validate and test documents
TX, Ty, Tqids, Tcids, Ttrue = load_data(TRAIN_DIR)
VX, Vy, Vqids, Vcids, Vtrue = load_data(VALI_DIR)
EX, Ey, Eqids, Ecids, Etrue = load_data(TEST_DIR)

# Show some information about the shapes
print("TX shape:", TX.shape)
print("Ty shape:", Ty.shape)
print("Tqids shape:", Tqids.shape)
print("Tcids shape:", Tcids.shape)
print("Ttrue shape:", Ttrue.shape)
print()
print("VX shape:", VX.shape)
print("Vy shape:", Vy.shape)
print("Vqids shape:", Vqids.shape)
print("Vcids shape:", Vcids.shape)
print("Vtrue shape:", Vtrue.shape)
print()
print("EX shape:", EX.shape)
print("Ey shape:", Ey.shape)
print("Eqids shape:", Eqids.shape)
print("Ecids shape:", Ecids.shape)
print("Etrue shape:", Etrue.shape)

TX shape: (160879, 3)
Ty shape: (160879,)
Tqids shape: (160879,)
Tcids shape: (160879,)
Ttrue shape: (2625, 27760)

VX shape: (12976, 3)
Vy shape: (12976,)
Vqids shape: (12976,)
Vcids shape: (12976,)
Vtrue shape: (213, 27760)

EX shape: (18951, 3)
Ey shape: (18951,)
Eqids shape: (18951,)
Ecids shape: (18951,)
Etrue shape: (312, 27760)


## Evaluation metrics

For evaluating the quality of results, we will use two metrics: [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain#Normalized_DCG) which is based on ranked suggestions (if the top ranked suggestion is correct it will contribute more to the score than getting the 2nd or 3rd suggestion right) and [F1 score](https://en.wikipedia.org/wiki/F1_score) that only considers binary choices: either a concept is a subject of a document or it is not, there are no fuzzy options. 

For the **NDCG score**, we can simply use the implementation from the `pyltr` library. Here we limit the NDCG evaluation to the top 100 most highly ranked subjects, meaning that subjects ranked 101st, 101nd etc won't contribute to the score (their effect would be almost negligible anyway do to the discounting performed in NDCG calculations).

For the **F1 score**, we can use the `scikit-learn` implementation. However, its usage is complicated by the fact that we are mostly dealing with ranked suggestions in the pyltr format, so we need to convert those into the multilabel prediction format understood by scikit-learn and further binarize the predictions using a thresholding strategy. For simplicity we will pick the top 5 suggested concepts, so we get an amount of subjects that is similar to the librarian-assigned ones (around 4.5 subjects per document), and compare those to the gold standard. The score for each document will be calculated individually and the final result will be the average of those scores (i.e. sample based average).

Finally, we will define an evaluation function that is given ranked suggestions as well as gold standard subjects (in both pyltr and scikit-learn formats) and some auxiliary information (qids and cids). It will calculate both NDCG and F1 score, print them out, and also return the scores so that they can be stored for later plotting etc.

In [8]:
import collections
from sklearn.metrics import f1_score

# initialize the NDCG metric implementation
ndcg_metric = pyltr.metrics.NDCG(k=100)

def ranking_to_multilabel(pred, qids, cids):
    """convert from pyltr ranking prediction format to sklearn multilabel prediction format"""
    doc_labels = collections.OrderedDict()  # key: qid, val: NumPy 1D array with length n_concepts
    for score, qid, cid in zip(pred, qids, cids):
        if qid not in doc_labels:
            doc_labels[qid] = np.zeros(n_concepts)
        doc_labels[qid][cid] = score
    return np.array(list(doc_labels.values()))

def binarize_multilabel_pred(pred, k=5):
    """convert predictions with scores to a boolean matrix, taking the top K predictions per document"""
    # I'm pretty sure this could be done easier, perhaps in a single NumPy operation, but I can't figure it out
    # Using a loop instead, it's not going to take long anyway
    binary_labels = np.zeros(pred.shape, dtype=np.bool)
    pred_order = pred.argsort()[:,::-1]
    for i, mask in enumerate(pred_order[:,:k]):
        binary_labels[i,mask] = True
    return binary_labels

def evaluate(name, qids, y, y_true, pred, cids):
    """evaluate a ranking, calculating its NDCG and F1 score. Print and return the scores"""
    # calculate NDCG score
    ndcg = ndcg_metric.calc_mean(qids, y, pred)

    # calculate F1 score
    pred_labels = ranking_to_multilabel(pred, qids, cids)
    f1 = f1_score(y_true, binarize_multilabel_pred(pred_labels), average='samples')
    
    print("{:>20}:\tNDCG={:.4f}\tF1={:.4f}".format(name, ndcg, f1))
    return ndcg, f1

## Baseline evaluation

Now we are ready for some baseline evaluations: we can evaluate the individual algorithms as well as a simplistic combination method using the mean of assigned scores (the only one implemented in Annif v0.31.0). We will store the results for later too.

In [12]:
# initialize dictionary for collecting evaluation results
eval_results = collections.OrderedDict()

# evaluate the individual algorithms
for algoid, name in enumerate(ALGORITHMS):
    eval_results[name] = evaluate(name, Eqids, Ey, Etrue, EX[:,algoid], Ecids)

# evaluate the combination of algorithms using mean of scores
eval_results["mean"] = evaluate("mean", Eqids, Ey, Etrue, EX.mean(1), Ecids)

               tfidf:	NDCG=0.4907	F1=0.2220
            fasttext:	NDCG=0.3865	F1=0.1329
                maui:	NDCG=0.3945	F1=0.1861
                mean:	NDCG=0.5432	F1=0.2632


We can see that out of individual algorithms, TF-IDF performed best, followed by Maui and fastText. The mean of scores gave better results than any algorithm alone. The NDCG and F1 measures seem to agree on which methods are better than others, as they both ranked the methods in the same order.

Note that the F1 scores may seem quite low (in some classification tasks F1 scores of 0.95 and more are not uncommon), but keep in mind that the task given to the algorithms is quite difficult: for each document, it should pick the 5 subjects (out of nearly 28000) that best describe that document. There are numerous ways of getting this wrong and only a few ways of picking a good set of subjects. So even a rather low F1 score of 0.2 means that around one out of five subjects was right - much better than chance.

## TODO: union-of-top-K
## TODO: mean-of-normalized-scores (L1, L2, max)
## TODO: PAV aka Isotonic Regression

## Learning to Rank

Let's try a true machine learning based fusion approach. We will apply the state of the art Learning to Rank algorithm LambdaMART, which is typically used for ranking results in a search engine. In this case, we consider each document a "query" and the predicted subjects as "results" and will try to come up with the ideal ranking of those subjects so that gold standard subjects are ranked as high as possible while non-relevant subjects are ranked lower. The algorithm is given a ranking measure that it then attempts to optimize; we will use NDCG as the measure to optimize.

A Learning to Rank algorithm requires features for the "results" (i.e. subjects in our case); here we will simply use the raw scores predicted by the subject indexing algorithms as the features.

The LambdaMART algorithm is conveniently implemented in the `pyltr` library. This implementation has some advanced features: it can be connected to a monitor that periodically evaluates the learned model on validation data and keeps track of the learning progress. If the monitor notices that the learning has reached a plateau - no improvement in validation scores has been made in the last K training rounds - it can stop the learning as well as roll back the model to the state when it reached that plateau. This is called *early stopping* and *trimming*, respectively, and it should help guard against overfitting the model on the training data.

In [13]:
# create a monitor for early stopping and trimming, using validation data
monitor = pyltr.models.monitors.ValidationMonitor(
    VX, Vy, Vqids, metric=ndcg_metric, stop_after=20)

# create a LambdaMART model for learning a suitable ranking
model = pyltr.models.LambdaMART(
    metric=ndcg_metric,
    n_estimators=100,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)

# fit the model to training data
model.fit(TX, Ty, Tqids, monitor=monitor)

 Iter  Train score    Remaining                           Monitor Output 
    1 2485059027827.9087       67.92m      C:      0.6158 B:      0.6158 S:  0
    2 2485059027827.9072       65.77m      C:      0.6111 B:      0.6158 S:  1
    3 2357295130178.3208       65.16m      C:      0.6114 B:      0.6158 S:  2
    4 2357295130178.3242       64.39m      C:      0.6130 B:      0.6158 S:  3
    5 2357295130178.3257       63.20m      C:      0.6130 B:      0.6158 S:  4
    6 2357295130178.3286       62.13m      C:      0.6139 B:      0.6158 S:  5
    7 2357295130178.3286       61.30m      C:      0.6130 B:      0.6158 S:  6
    8 2357295130178.3335       60.66m      C:      0.6159 B:      0.6159 S:  0
    9 2485059027827.9248       59.84m      C:      0.6191 B:      0.6191 S:  0
   10 2485059027827.9248       59.01m      C:      0.6187 B:      0.6191 S:  1
   15 2485059027827.9507       55.28m      C:      0.6356 B:      0.6356 S:  0
   20 2679180801228.5591       51.94m      C:      0.6516

Now we have a model trained on the training documents and validated on the validation documents. First, since we spent so much time creating the model, let's save it for later:

In [15]:
# Save the learned model for later use
from sklearn.externals import joblib
joblib.dump(model, 'ltr-model.joblib') 

['ltr-model.joblib']

Let's use the model to predict subject rankings on the evaluation documents and evaluate how well it did compared to the other fusion approaches.

In [16]:
Epred = model.predict(EX)
eval_results["ltr"] = evaluate("ltr", Eqids, Ey, Etrue, Epred, Ecids)

                 ltr:	NDCG=0.5802	F1=0.2795


We can also check how much the input from each algorithm contributed to the rankings.

In [18]:
for feature, importance in zip(ALGORITHMS, model.feature_importances_):
    print("{:>20} importance: {:.4f}".format(feature, importance))

               tfidf importance: 0.2383
            fasttext importance: 0.3477
                maui importance: 0.4139


## TODO: plot the results using matplotlib